In [1]:
import ciropt as co
import cvxpy as cp

import PEPit
import PEPit.functions as pep_func
from PEPit.constraint import Constraint as pep_constr
from PEPit.primitive_steps import proximal_step as pep_proximal_step

import ciropt.function as co_func
import ciropt.function as co_func
from ciropt.constraint import Constraint as co_constr
import sympy as sp

In [2]:
R = 0.8
L_smooth = [100, 100]
mu = [0., 2]
L = 2
C = 15

solver = "ipopt"
# solver = "ipopt_qcqp"
# solver = "ipopt_qcqp_matrix"

# Ciropt problem

In [3]:
def dadmm_C_graph6(mus, L_smooths, params=None):
    # graph with 6 nodes and 7 edges (1,2), (1,3), (2,3), (2,4), (3,4), (4,5), (4,6)
    # 1 -- 2 -- 4 -- 5
    #   \  |  /  \
    #      3       6 
    # D-ADMM with Capacitor at e_45
    if params is not None:
        # verification mode: PEP
        problem = PEPit.PEP()
        package = pep_func 
        Constraint = pep_constr
        proximal_step = pep_proximal_step 
        h, eta, rho, gamma, C, invC, L, invL, R, invR = params["h"], params["eta"], params["rho"], params["gamma"], \
                                        params["C"], params["invC"], params["L"], params["invL"], params["R"], params["invR"]
    else:
        # Ciropt mode
        problem = co.CircuitOpt()
        package = co_func
        Constraint = co_constr
        proximal_step = co_func.proximal_step 
        C, R, L = sp.symbols('C'), sp.symbols('R'), sp.symbols('L')
        invC, invL, invR = sp.symbols("invC"), sp.symbols("invL"), sp.symbols("invR")
        problem.discretization_params = sorted(problem.discretization_params + 
                                               ["C", "invC", "L", "invL", "R", "invR"])
        h, eta, rho, gamma = problem.h, problem.eta, problem.rho, problem.gamma

    f1 = co.define_function(problem, mus[0], L_smooths[0], package)
    f2 = co.define_function(problem, mus[0], L_smooths[0], package)
    f3 = co.define_function(problem, mus[0], L_smooths[0], package)
    f4 = co.define_function(problem, mus[1], L_smooths[1], package)
    f5 = co.define_function(problem, mus[1], L_smooths[1], package)
    f6 = co.define_function(problem, mus[0], L_smooths[0], package)

    x_star, y_star, f_star = (f1 + f2 + f3 + f4 + f5 + f6).stationary_point(return_gradient_and_function_value=True)
    y1_star, f1_star = f1.oracle(x_star)
    y2_star, f2_star = f2.oracle(x_star)
    y3_star, f3_star = f3.oracle(x_star)
    y4_star, f4_star = f4.oracle(x_star)
    y5_star, f5_star = f5.oracle(x_star)
    y6_star, f6_star = f6.oracle(x_star)
    # when f is not differentiable
    problem.add_constraint(Constraint((y1_star + y2_star + y3_star + y4_star + y5_star + y6_star - y_star) ** 2, "equality"))

    y1_12_star = problem.set_initial_point()
    y1_13_star = y1_star - y1_12_star
    y2_23_star = problem.set_initial_point()
    y2_24_star = y2_star + y1_12_star - y2_23_star
    y3_34_star = y3_star + y1_13_star + y2_23_star
    # y5_54_star = y5_star;     y6_64_star = y6_star
    # currents on each new at equilibrium sum to 0
    problem.add_constraint(Constraint((y2_24_star + y3_34_star + y5_star + y6_star + y4_star) ** 2, "equality"))

    # edges (1,2), (1,3), (2,3), (2,4), (3,4), (4,5), (4,6)
    e_12_1 = problem.set_initial_point()
    e_13_1 = problem.set_initial_point()
    e_23_1 = problem.set_initial_point()
    e_24_1 = problem.set_initial_point()
    e_34_1 = problem.set_initial_point()
    e_45_1 = problem.set_initial_point()
    e_46_1 = problem.set_initial_point()
    # initialize currents on inductors to sum to 0 on every edge
    i_L_12_1 = problem.set_initial_point()
    i_L_13_1 = problem.set_initial_point()
    i_L_23_1 = problem.set_initial_point()
    i_L_24_1 = problem.set_initial_point()
    i_L_34_1 = problem.set_initial_point()
    i_L_45_1 = problem.set_initial_point()
    i_L_54_1 = problem.set_initial_point()
    i_L_46_1 = problem.set_initial_point()


    x1_2, y1_2, f1_2 = proximal_step((1/2)*((R * i_L_12_1 + e_12_1) + (R * i_L_13_1 + e_13_1)), f1, R/2)
    x2_2, y2_2, f2_2 = proximal_step((1/3)*((-R * i_L_12_1 + e_12_1) + (R * i_L_23_1 + e_23_1) \
                                            + (R * i_L_24_1 + e_24_1)), f2, R/3)
    x3_2, y3_2, f3_2 = proximal_step((1/3)*((-R * i_L_13_1 + e_13_1) + (-R * i_L_23_1 + e_23_1) \
                                            + (R * i_L_34_1 + e_34_1)), f3, R/3)
    x4_2, y4_2, f4_2 = proximal_step((1/4)*((-R * i_L_24_1 + e_24_1) + (-R * i_L_34_1 + e_34_1) \
                                            + (R * i_L_45_1 + e_45_1) + (R * i_L_46_1 + e_46_1)), f4, R/4)
    x5_2, y5_2, f5_2 = proximal_step(R * i_L_54_1 + e_45_1, f5, R)
    x6_2, y6_2, f6_2 = proximal_step(-R * i_L_46_1 + e_46_1, f6, R)

    e_12_2 = (x1_2 + x2_2) / 2
    e_13_2 = (x1_2 + x3_2) / 2
    e_23_2 = (x2_2 + x3_2) / 2
    e_24_2 = (x2_2 + x4_2) / 2
    e_34_2 = (x3_2 + x4_2) / 2
    e_45_2 = e_45_1 - ( h * invC) * (i_L_45_1 + i_L_54_1 + (2 * e_45_1 - x4_2 - x5_2) * invR)
    e_46_2 = (x4_2 + x6_2) / 2
    i_L_12_2 = i_L_12_1 + ( h * invL) * (e_12_2 - x1_2)      
    i_L_13_2 = i_L_13_1 + ( h * invL) * (e_13_2 - x1_2)
    i_L_23_2 = i_L_23_1 + ( h * invL) * (e_23_2 - x2_2)
    i_L_24_2 = i_L_24_1 + ( h * invL) * (e_24_2 - x2_2)
    i_L_34_2 = i_L_34_1 + ( h * invL) * (e_34_2 - x3_2)
    i_L_45_2 = i_L_45_1 + ( h * invL) * (e_45_2 - x4_2)
    i_L_54_2 = i_L_54_1 + ( h * invL) * (e_45_2 - x5_2)
    i_L_46_2 = i_L_46_1 + ( h * invL) * (e_46_2 - x4_2)
    
    # energy of two inductors on each net is twice the energy on one inductor
    E_1 = gamma * ((e_12_1 - x_star)**2 + (e_13_1 - x_star)**2 + (e_23_1 - x_star)**2 + (e_24_1 - x_star)**2\
                   + (e_34_1 - x_star)**2 + (e_46_1 - x_star)**2) +  \
            + L * (i_L_12_1 - y1_12_star) ** 2 + L * (i_L_13_1 - y1_13_star) ** 2 \
            + L * (i_L_23_1 - y2_23_star) ** 2 + L * (i_L_24_1 - y2_24_star) ** 2 \
            + L * (i_L_34_1 - y3_34_star) ** 2 \
            + (L/2) * (i_L_45_1 + y5_star) ** 2 + (L/2) * (i_L_54_1 - y5_star) ** 2 \
            + L * (i_L_46_1 + y6_star) ** 2 \
            + (C / 2) * (e_45_1 - x_star) ** 2
    E_2 = gamma * ((e_12_2 - x_star)**2 + (e_13_2 - x_star)**2 + (e_23_2 - x_star)**2 + (e_24_2 - x_star)**2\
                   + (e_34_2 - x_star)**2 + (e_46_2 - x_star)**2) +  \
            + L * (i_L_12_2 - y1_12_star) ** 2 + L * (i_L_13_2 - y1_13_star) ** 2 \
            + L * (i_L_23_2 - y2_23_star) ** 2 + L * (i_L_24_2 - y2_24_star) ** 2 \
            + L * (i_L_34_2 - y3_34_star) ** 2 \
            + (L/2) * (i_L_45_2 + y5_star) ** 2 + (L/2) * (i_L_54_2 - y5_star) ** 2 \
            + L * (i_L_46_2 + y6_star) ** 2 \
            + (C / 2) * (e_45_2 - x_star) ** 2
    # currents on resistors on each net sum to 0
    Delta_2 = rho * ((2*invR) * ((e_12_2 - x1_2)**2 + (e_13_2 - x3_2)**2 \
                         + (e_23_2 - x2_2)**2 + (e_24_2 - x2_2)**2 \
                         + (e_34_2 - x4_2)**2  + (e_46_2 - x6_2)**2 )  \
                    + (1*invR) * ((e_45_2 - x5_2)**2 + (e_45_2 - x4_2)**2)) \
              + eta * ( f1_2 - f1_star - y1_star * (x1_2 - x_star) \
                    + f2_2 - f2_star - y2_star * (x2_2 - x_star) \
                    + f3_2 - f3_star - y3_star * (x3_2 - x_star) \
                    + f4_2 - f4_star - y4_star * (x4_2 - x_star) \
                    + f5_2 - f5_star - y5_star * (x5_2 - x_star) \
                    + f6_2 - f6_star - y6_star * (x6_2 - x_star))

    problem.set_performance_metric(E_2 - (E_1 - Delta_2))
    return problem


In [4]:
print(f"{mu=}, {L_smooth=}, {R=}, {C=}, {L=}, {solver=}")

mu=[0.0, 2], L_smooth=[100, 100], R=0.8, C=15, L=2, solver='ipopt'


In [5]:
problem = dadmm_C_graph6( mu, L_smooth)
problem.obj = problem.eta + problem.rho * 0.1

bounds = {"C":{"ub": 20, "lb":12},
          "R":{"ub": 0.9, "lb":0.5},
          "L":{"ub": 3, "lb":1}}

res, sol = problem.solve(solver=solver, extra_dim=530, bounds=bounds, verbose=False)[:2]

dim_G=29, dim_F=12
Ipopt total # of variables = 1011
Actual # of variables = 474


In [6]:
res

{'C': 20.000000185323408,
 'L': 3.000000027632129,
 'R': 0.5006518814170846,
 'eta': 5.8946161576573015,
 'gamma': 11.263722454630313,
 'h': 5.407711160894115,
 'invC': 0.049999999536691736,
 'invL': 0.3333333302630969,
 'invR': 1.9973958695002236,
 'rho': -7.808308896867274e-09}

In [7]:
# {'C': 18.00000016422373,
#  'L': 3.0000000272145817,
#  'R': 0.5432001487867559,
#  'eta': 5.586517803570342,
#  'gamma': 9.851337233500212,
#  'h': 5.141722689015985,
#  'invC': 0.05555555504869218,
#  'invL': 0.3333333303094909,
#  'invR': 1.8409420583435165,
#  'rho': 9.279396912061786e-09}

# {'C': 18.00000017205779,
#  'L': 2.0000000000021845,
#  'R': 0.49999999082527685,
#  'eta': 4.553025144693309,
#  'gamma': 8.722547253422393,
#  'h': 4.198902827245454,
#  'invC': 0.055555555024513016,
#  'invL': 0.4999999999999984,
#  'invR': 2.000000036698893,
#  'rho': -3.066024650469293e-09}


# {'C': 15.000000143594386,
#  'L': 2.0,
#  'R': 0.49999999367453674,
#  'eta': 4.174822294507111,
#  'gamma': 7.986324661697676,
#  'h': 3.8366031716977202,
#  'invC': 0.06666666602846939,
#  'invL': 0.5,
#  'invR': 2.0000000253018535,
#  'rho': -3.3691535584123036e-09}

# {'C': 18.00000016252312,
#  'L': 2.0000000000000013,
#  'R': 0.7999999999999993,
#  'eta': 3.882325611200754,
#  'gamma': 4.700910522843513,
#  'h': 3.6976844281294214,
#  'invC': 0.05555555505394075,
#  'invL': 0.4999999999999997,
#  'invR': 1.2500000000000038,
#  'rho': 1.7494454375448952e-09}

In [8]:
1/15

0.06666666666666667

# PEP verification

In [9]:
res["rho"] = 0

In [10]:
problem = dadmm_C_graph6( mu, L_smooth, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.0897382649033283e-09
